In [1]:
!pip install pinecone-client cohere sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [5]:
import os
from pinecone import Pinecone, ServerlessSpec

# Create a Pinecone instance with your API key
pc = Pinecone(api_key='your-pinecone-api-key')

# Check if the index exists
index_name = 'document-index'
if index_name not in pc.list_indexes().names():
    # Create the index if it doesn't exist
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the index
index = pc.Index(index_name)

# Print confirmation
print(f"Connected to Pinecone index: {index_name}")


Index 'document-index' already exists.
Connected to Pinecone index: document-index


In [6]:
from sentence_transformers import SentenceTransformer

# Load the Sentence-Transformer model
model = SentenceTransformer('all-mpnet-base-v2')


documents = [
    "/content/corpus.txt",

]

# Generate embeddings for the documents
document_embeddings = model.encode(documents)

# Upsert the document embeddings into Pinecone
for i, embedding in enumerate(document_embeddings):
    index.upsert([{
        'id': f'doc-{i}',
        'values': embedding.tolist(),
        'metadata': {'text': documents[i]}
    }])

print("Documents successfully upserted into Pinecone!")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Documents successfully upserted into Pinecone!


In [7]:
def retrieve_documents(query):
    # Generate query embedding using the new model
    query_embedding = model.encode([query])[0]

    # Query Pinecone for the top 3 matches
    result = index.query(
        vector=query_embedding.tolist(),
        top_k=3,
        include_metadata=True
    )

    # Combine the document texts to form the context
    context = " ".join([match['metadata']['text'] for match in result['matches']])

    return context


In [11]:
import cohere

# Initialize Cohere
cohere_client = cohere.Client('your-cohere-api-key')

def generate_answer(query, context):
    # Create a prompt using the retrieved context
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"

    # Use Cohere to generate the answer
    response = cohere_client.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=150,
        temperature=0.7
    )

    return response.generations[0].text.strip()


In [12]:
# Predefined queries
predefined_queries = [
    "Can you tell me about the 2022 Chardonnay?",
    "What white wines do you have?",
   
]

# Loop through each predefined query
for query in predefined_queries:
    print(f"Query: {query}")

    # Retrieve relevant documents from Pinecone
    context = retrieve_documents(query)

    if not context:
        print("No relevant documents found for this query.")
        continue

    # Generate an answer using the retrieved context
    answer = generate_answer(query, context)

    # Print the result
    print(f"Answer: {answer}\n")


Query: Can you tell me about the 2022 Chardonnay?
Answer: I have searched the provided corpus and found the following information about the 2022 Chardonnay:

The 2022 vintage of our Chardonnay is a true expression of the unique terroir of our vineyard. This wine showcases a brilliant straw color with hints of green, inviting you to take a sip. On the nose, it greets you with delightful aromas of fresh citrus, ripe melon, and subtle notes of vanilla.

In the mouth, the 2022 Chardonnay is a well-rounded delight. It offers a harmonious balance of crisp acidity and a creamy texture, leaving a refreshing sensation. Flavors of green apple, pear, and a touch of oak spice linger on the palate, creating a long and satisfying finish

Query: What white wines do you have?
Answer: The wine list is not provided, so it is not possible to determine which white wines are available. However, some popular white wine varieties include Chardonnay, Sauvignon Blanc, Pinot Grigio, Riesling, and Moscato.

